In [12]:
#Map the dataset to the FHIR resources
import pandas as pd
import json
import uuid
from fhir.resources.bundle import Bundle, BundleEntry, BundleEntryRequest
from fhir.resources.medicationrequest import MedicationRequest
from fhir.resources.reference import Reference
from fhir.resources.codeablereference import CodeableReference
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.dosage import Dosage
from pydantic import ValidationError

# Load the CSV file into a DataFrame
csv_file_path = 'medications_on_discharge_4696_main.csv'
data = pd.read_csv(csv_file_path)

# Define chunk size for batching the resources
chunk_size = 500

# Convert DataFrame rows to list of tuples (index, row)
data_list = list(data.iterrows())

# Define a function to split lists into chunks
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Create a list to hold Bundle entries
for i, chunk in enumerate(chunk_list(data_list, chunk_size)):
    entries = []
    
    for index, row in chunk:
        try:
            # Create a unique identifier for each resource
            resource_id = str(uuid.uuid4())  # Generate a UUID

            # Determine display name for category
            display_name = {
                "inpatient": "Inpatient",
                "community": "Community",
                "outpatient": "Outpatient"
            }.get(row["category"])  # Get the display name or None if not found

            # Check for required fields and handle missing values
            if pd.isna(row["server_id"]) or pd.isna(row["medication_name"]):
                print(f"Skipping row {index} due to missing required fields.")
                continue

            # Prepare medication field
            rxnorm_code = str(row["rxnorm_code"]).strip() if pd.notna(row["rxnorm_code"]) else ""
            if rxnorm_code and rxnorm_code != "":
                # Remove trailing ".0" if present
                if rxnorm_code.endswith(".0"):
                    rxnorm_code = rxnorm_code[:-2]

                medication = CodeableReference(
                    concept=CodeableConcept(
                        coding=[Coding(system="http://www.nlm.nih.gov/research/umls/rxnorm", code=rxnorm_code, display=row["medication_name"])]
                    )
                )
            else:
                # Use medication name as text if code is not available
                medication = CodeableReference(
                    concept=CodeableConcept(
                        text=row["medication_name"]
                    )
                )
            
            # Prepare dosageInstruction field
            dosage_parts = [str(row["dose"]).strip(), str(row["route"]).strip(), str(row["frequency"]).strip()]
            # Remove any 'nan' strings from dosage parts
            dosage_parts = [part for part in dosage_parts if part and part.lower() != 'nan']
            dosage_instruction = [Dosage(text=" ".join(dosage_parts))] if dosage_parts else None
            
            # Prepare reason field
            reason = None
            if pd.notna(row["diagnosis"]):
                reason = [CodeableReference(concept=CodeableConcept(text=row["diagnosis"]))]
            
            # Create a new MedicationRequest resource
            medication_request = MedicationRequest(
                id=resource_id,
                status="active",
                intent=row["intent"],
                subject=Reference(reference=f'Patient/{row["server_id"]}'),
                medication=medication,  # Use the prepared medication field
                text={
                    "status": "generated",
                    "div": f"<div xmlns='http://www.w3.org/1999/xhtml'>MedicationRequest record for Patient/{row['server_id']}</div>"
                }
            )

            # Add dosageInstruction if present
            if dosage_instruction:
                medication_request.dosageInstruction = dosage_instruction
            
            # Add reason if present
            if reason:
                medication_request.reason = reason
            
            # Map category to MedicationRequest.category if there's a display name
            if pd.notna(row["category"]) and display_name:
                medication_request.category = [
                    CodeableConcept(
                        coding=[
                            Coding(
                                system="http://terminology.hl7.org/CodeSystem/medicationrequest-admin-location",
                                code=row["category"],
                                display=display_name
                            )
                        ]
                    )
                ]
            
            # Create a Bundle entry
            entry = BundleEntry(
                fullUrl=f"urn:uuid:{resource_id}",
                resource=medication_request,
                request=BundleEntryRequest(
                    method="POST",
                    url="MedicationRequest"
                )
            )
            
            # Add the entry to the list
            entries.append(entry)
            
        except ValidationError as e:
            print(f"Validation error for row {index}: {e}")
        except ValueError as e:
            print(f"Value error for row {index}: {e}")

    # Create a Bundle resource for this chunk
    bundle = Bundle(
        resourceType="Bundle",
        type="transaction",
        entry=entries
    )
    
    # Convert the Bundle to JSON
    bundle_json = bundle.dict()
    
    # Save the Bundle JSON to a file
    output_file = f'fhir_medication_requests_bundle_chunk_{i+1}.json'
    with open(output_file, 'w') as f:
        json.dump(bundle_json, f, indent=2)
    
    print(f"FHIR MedicationRequest Bundle chunk {i+1} saved to {output_file}")


FHIR MedicationRequest Bundle chunk 1 saved to fhir_medication_requests_bundle_chunk_1.json
FHIR MedicationRequest Bundle chunk 2 saved to fhir_medication_requests_bundle_chunk_2.json
FHIR MedicationRequest Bundle chunk 3 saved to fhir_medication_requests_bundle_chunk_3.json
FHIR MedicationRequest Bundle chunk 4 saved to fhir_medication_requests_bundle_chunk_4.json
FHIR MedicationRequest Bundle chunk 5 saved to fhir_medication_requests_bundle_chunk_5.json
FHIR MedicationRequest Bundle chunk 6 saved to fhir_medication_requests_bundle_chunk_6.json
FHIR MedicationRequest Bundle chunk 7 saved to fhir_medication_requests_bundle_chunk_7.json
FHIR MedicationRequest Bundle chunk 8 saved to fhir_medication_requests_bundle_chunk_8.json
FHIR MedicationRequest Bundle chunk 9 saved to fhir_medication_requests_bundle_chunk_9.json
FHIR MedicationRequest Bundle chunk 10 saved to fhir_medication_requests_bundle_chunk_10.json


In [13]:
#Uploading the resources to the local HAPI FHIR server (R5)
import json
import requests
import os

# Define the HAPI FHIR server URL
url = 'http://localhost:8080/fhir'

# Define the base file name pattern
file_pattern = 'fhir_medication_requests_bundle_chunk_{}.json'

# Define the number of files
num_files = 10

# List to hold all resource IDs
all_resource_ids = []

# Process each file sequentially
for i in range(1, num_files + 1):
    # Construct the file name
    input_file_path = file_pattern.format(i)
    
    # Check if the file exists
    if not os.path.exists(input_file_path):
        print(f"File {input_file_path} does not exist. Skipping.")
        continue
    
    # Load the JSON data from the file
    with open(input_file_path, 'r') as file:
        data = json.load(file)
    
    # Make a POST request to upload the Bundle
    response = requests.post(url, json=data, headers={'Content-Type': 'application/fhir+json'})
    
    if response.status_code == 200:
        print(f"Successfully uploaded {input_file_path} to the HAPI FHIR server.")
        
        # Parse the response to extract resource IDs
        response_data = response.json()
        
        for entry in response_data.get('entry', []):
            resource_id = entry.get('response', {}).get('location', '')
            all_resource_ids.append(resource_id)
    else:
        print(f"Failed to upload {input_file_path}. Status code: {response.status_code}")
        print(response.text)

# Save all resource IDs to a new JSON file
output_file_path = 'uploaded_resource_ids.json'
with open(output_file_path, 'w') as output_file:
    json.dump(all_resource_ids, output_file, indent=4)

print(f"Resource IDs from all files have been saved to {output_file_path}.")


Successfully uploaded fhir_medication_requests_bundle_chunk_1.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_2.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_3.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_4.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_5.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_6.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_7.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_8.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_9.json to the HAPI FHIR server.
Successfully uploaded fhir_medication_requests_bundle_chunk_10.json to the HAPI FHIR server.
Resource IDs from all files have been saved to uploaded_resource_ids.json.
